In [3]:
! pip install pyspark

In [4]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [5]:
spark = SparkSession.builder.getOrCreate()

In [6]:
clientes = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/clientes.csv', header=True, inferSchema=True)
itens_pedido = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/itens_pedido.csv', header=True, inferSchema=True)
pagamentos_pedido = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/pagamentos_pedido.csv', header=True, inferSchema=True)
pedidos = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/pedidos.csv', header=True, inferSchema=True)

In [7]:
clientes.show(5)
itens_pedido.show(5)
pagamentos_pedido.show(5)
pedidos.show(5)

+--------------------+--------------------+-----------+--------------------+--------------+
|          id_cliente|    id_unico_cliente|cep_cliente|      cidade_cliente|estado_cliente|
+--------------------+--------------------+-----------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|      14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|       9790|sao bernardo do c...|            SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|       1151|           sao paulo|            SP|
|b2b6027bc5c5109e5...|259dac757896d24d7...|       8775|     mogi das cruzes|            SP|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|      13056|            campinas|            SP|
+--------------------+--------------------+-----------+--------------------+--------------+
only showing top 5 rows

+--------------------+--------------+--------------------+--------------------+-------------------+-----+-----------+
|           id_pedido|item_id

In [8]:
# quantos estados diferentes estão presentes na tabela clientes?
clientes.agg(countDistinct(col('estado_cliente')).alias('qtd_estados_distintos')).show()

+---------------------+
|qtd_estados_distintos|
+---------------------+
|                   27|
+---------------------+



In [9]:
# Qual é a média e a variância do valor_pagamento para cada tipo de pagamento?
pagamentos_pedido.groupBy('tipo_pagamento').agg(
    avg('valor_pagamento').alias('pagamento_medio'),
    var_samp('valor_pagamento').alias('pagamento_variancia')
).show()

+--------------+------------------+-------------------+
|tipo_pagamento|   pagamento_medio|pagamento_variancia|
+--------------+------------------+-------------------+
|        boleto| 145.0344354023452| 45616.869821012544|
|   not_defined|               0.0|                0.0|
|   credit_card|163.31902063935797|  49336.98820414979|
|       voucher| 65.70335411255421| 13344.682202516573|
|    debit_card|142.57017004578165| 60414.395995128165|
+--------------+------------------+-------------------+



In [10]:
# Média e variância do valor dos pagamentos arredondado, agrupados por Tipo de Pagamento
pagamentos_pedido.groupBy('tipo_pagamento').agg(
    format_number(avg('valor_pagamento'),2).alias('pagamento_medio'),
    format_number(var_samp('valor_pagamento'),2).alias('pagamento_variancia')
).show()

+--------------+---------------+-------------------+
|tipo_pagamento|pagamento_medio|pagamento_variancia|
+--------------+---------------+-------------------+
|        boleto|         145.03|          45,616.87|
|   not_defined|           0.00|               0.00|
|   credit_card|         163.32|          49,336.99|
|       voucher|          65.70|          13,344.68|
|    debit_card|         142.57|          60,414.40|
+--------------+---------------+-------------------+



In [11]:
pedidos.printSchema()

root
 |-- id_pedido: string (nullable = true)
 |-- id_cliente: string (nullable = true)
 |-- status_pedido: string (nullable = true)
 |-- data_compra_pedido: timestamp (nullable = true)
 |-- data_aprovacao_pedido: timestamp (nullable = true)
 |-- data_envio_transportadora: timestamp (nullable = true)
 |-- data_entrega_cliente: timestamp (nullable = true)
 |-- data_estimada_entrega: timestamp (nullable = true)



In [16]:
# Exiba as datas do primeiro e do último pedido para cada um dos status
pedidos_ordenados = pedidos.orderBy('data_compra_pedido')

In [18]:
datas_compras_df = pedidos_ordenados.groupBy('status_pedido').agg(
    first('data_compra_pedido').alias('primeiro'),
    last('data_compra_pedido').alias('ultimo')
)

datas_compras_df.show()

+-------------+-------------------+-------------------+
|status_pedido|           primeiro|             ultimo|
+-------------+-------------------+-------------------+
|      shipped|2016-09-04 21:15:19|2018-09-03 09:06:57|
|     canceled|2016-09-05 00:15:34|2018-10-17 17:30:18|
|     approved|2017-02-06 20:18:17|2017-04-25 01:25:34|
|     invoiced|2016-10-04 13:02:10|2018-08-14 18:45:08|
|      created|2017-11-06 13:12:34|2018-02-09 17:21:04|
|    delivered|2016-09-15 12:16:38|2018-08-29 15:00:37|
|  unavailable|2016-10-05 14:16:28|2018-08-21 12:21:00|
|   processing|2016-10-05 22:44:13|2018-07-23 18:03:03|
+-------------+-------------------+-------------------+



In [19]:
# Exibir o número de compras por ano em ordem ascendente
pedidos.withColumn('compra_anual', year('data_compra_pedido')).groupBy('compra_anual').count().orderBy(asc('compra_anual')).show()

+------------+-----+
|compra_anual|count|
+------------+-----+
|        2016|  329|
|        2017|45101|
|        2018|54011|
+------------+-----+



In [20]:
# Exibir o número de compras por ano em ordem descendente
pedidos.withColumn('compra_anual', year('data_compra_pedido')).groupBy('compra_anual').count().orderBy(desc('compra_anual')).show()

+------------+-----+
|compra_anual|count|
+------------+-----+
|        2018|54011|
|        2017|45101|
|        2016|  329|
+------------+-----+



In [21]:
itens_pedido.groupBy('id_pedido').agg(
    sum(col('preco') + col('valor_frete')),
    sum(col('preco') + col('valor_frete')).alias('valor_total_pedido'),
    format_number(sum(col('preco') + col('valor_frete')),2).alias('valor_total_pedido_formatado')
).show()

+--------------------+--------------------------+------------------+----------------------------+
|           id_pedido|sum((preco + valor_frete))|valor_total_pedido|valor_total_pedido_formatado|
+--------------------+--------------------------+------------------+----------------------------+
|014405982914c2cde...|         78.42999999999999| 78.42999999999999|                       78.43|
|019886de8f385a39b...|                     188.4|             188.4|                      188.40|
|01a6ad782455876aa...|                     50.09|             50.09|                       50.09|
|01d907b3e209269e1...|        169.76000000000002|169.76000000000002|                      169.76|
|028dc52e12ddda803...|                     61.72|             61.72|                       61.72|
|036dd381dfb3ec75e...|                     69.14|             69.14|                       69.14|
|03ebfa9712b7dbc70...|                     55.78|             55.78|                       55.78|
|0420da8d50a378401..

In [22]:
spark.stop()